In [8]:
#!pip install kagglehub
import kagglehub
import os
import shutil
from urllib.request import urlretrieve
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import pickle

#from datasketch import MinHash, MinHashLSH


In [9]:
def load_data():
    # Contains movieId, title, genres   
    ml_movies = pd.read_csv('data/ml-32m/movies.csv')
    
    # Contains userId, movieId, rating, timestamp
    ml_ratings = pd.read_csv('data/ml-32m/ratings.csv')
    
    # Contains userId, movieId, tag, timestamp
    ml_tags = pd.read_csv('data/ml-32m/tags.csv')

    # Contains movieId, imdbId, tmdbId
    ml_links = pd.read_csv('data/ml-32m/links.csv')

    # IMDB Dataset:
    imdb = pd.read_csv('data/imdb/movies.csv')

    return ml_movies, ml_ratings, ml_tags, ml_links, imdb

ml_movies, ml_ratings, ml_tags, ml_links, imdb = load_data()

/tmp/pbs.470611.hnode41/ipykernel_3272744/683732697.py:15: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb = pd.read_csv('data/imdb/movies.csv')


In [10]:
# Preprocessing IMDB:

#drop if description is "Add a plot" because we have no description then
imdb_descriptions = imdb[imdb['description'] != "Add a Plot"]

imdb_descriptions['id'] = imdb_descriptions['id'].str[2:]
imdb_descriptions['id'] = pd.to_numeric(imdb_descriptions['id'], errors='coerce')

#keep what we need
imdb_descriptions = imdb_descriptions[['id', 'description']]

#merge them, so we have a li nk between the two datasets
merged_links = ml_links.merge(imdb_descriptions, left_on='imdbId', right_on='id', how='inner').dropna(subset=['id'])

/tmp/pbs.470611.hnode41/ipykernel_3272744/493108502.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_descriptions['id'] = imdb_descriptions['id'].str[2:]
/tmp/pbs.470611.hnode41/ipykernel_3272744/493108502.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_descriptions['id'] = pd.to_numeric(imdb_descriptions['id'], errors='coerce')


In [11]:
#merge the movies with the descriptions if that is what we want
ml_movies_description = ml_movies.merge(merged_links, left_on='movieId', right_on='movieId', how='inner')

ml_movies_description
#ml_movies_description.to_csv('data/movies_with_description.csv', index=False)

,movieId,title,genres,imdbId,tmdbId,id,description
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,114709,A cowboy doll is profoundly threatened and jea...
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,113497,When two kids find and play a magical board ga...
2,6,Heat (1995),Action|Crime|Thriller,113277,949.0,113277,A group of high-end professional thieves start...
3,8,Tom and Huck (1995),Adventure|Children,112302,45325.0,112302,Two best friends witness a murder and embark o...
4,9,Sudden Death (1995),Action,114576,9091.0,114576,A former fireman takes on a group of terrorist...
...,...,...,...,...,...,...,...
51855,292541,The Settlers (2023),Drama|Western,10370812,989589.0,10370812,"A mixed-race Chilean, rides south on an expedi..."
51856,292585,Night Crawlers (2009),Comedy|Horror,985060,147230.0,985060,Blood is thicker than water in this tiny Texas...
51857,292605,Our River... Our Sky (2023),Drama|War,10676126,855800.0,10676126,Baghdad. The last week of 2006. All over the c...
51858,292613,Freelance (2023),Action|Comedy,15744298,897087.0,15744298,An ex special forces operator takes a job to p...


## Tokenize for BERT and inspection of description length in dataset in terms of characters, words and tokenized data

In [9]:
#run inpquts through the model in batches of 8 to avoid memory issues and also pass attention masks
import torch
from tqdm import tqdm
outputs = []
model = model.to("cuda")
inputs = inputs.to("cuda")
batch_size = 16
#without saving gradients to avoid gpu memory issues/leaks
# if outputs file exists, load it, else run the model
if os.path.exists('data/outputs.pt'):
    outputs = torch.load('data/outputs.pt')
else:
    with torch.no_grad():
        for i in tqdm(range(0, len(inputs['input_ids'])//8, batch_size)):
            batch = inputs['input_ids'][i:i+batch_size]
            attention_mask = inputs['attention_mask'][i:i+batch_size]
            output = model(batch, attention_mask=attention_mask).last_hidden_state
            #output = model(batch, attention_mask=attention_mask)['logits']

            #send the output to the cpu afterwards to avoid memory issues on the gpu
            outputs.append(output.detach().cpu())

    #make list of tensors into one tensor
    outputs = torch.cat(outputs)
    torch.save(outputs, 'data/outputs.pt')


  0%|          | 0/406 [00:00<?, ?it/s]

100%|██████████| 406/406 [01:12<00:00,  5.58it/s]


In [63]:
import numpy as np
embeddings = np.load("/work3/s204161/embeddings.npy")
embeddings = embeddings.reshape(embeddings.shape[0],embeddings.shape[1]*embeddings.shape[2])


In [98]:
from sklearn.cluster import MiniBatchKMeans

# dont standardize embeddings vector to have mean 0 and std 1 ... because for our hidden weights, some features ARE more important than others...
# embeddings = embeddings - embeddings.mean(axis=0)
# embeddings = embeddings / embeddings.std(axis=0)

# Define MiniBatch K-Means
mini_batch_kmeans = MiniBatchKMeans(n_clusters=200, batch_size=500, random_state=42)
mini_batch_kmeans.fit(embeddings)

# Get cluster labels
import copy
labels = copy.deepcopy(mini_batch_kmeans.labels_)
centers = copy.deepcopy(mini_batch_kmeans.cluster_centers_)


/work3/s204161/miniconda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


MiniBatchKMeans(batch_size=500, n_clusters=200, random_state=42)

In [ ]:
## for sparse clusters of length < 5 join them with neighbouring clusters
# get the number of movies in each cluster
unique, counts = np.unique(labels, return_counts=True)
cluster_counts = dict(zip(unique, counts))

# get the number of clusters
n_clusters = len(cluster_counts)
from tqdm import tqdm

# merge clusters with less than 5 movies
for cluster, count in tqdm(list(cluster_counts.items())):  # Iterate over a copy of the dictionary
    if count < 5:
        while True:  # Find a valid cluster to merge into
            distances = np.linalg.norm(centers - centers[cluster], axis=1)
            distances[cluster] = np.inf  # Avoid merging a cluster with itself
            closest_cluster = np.argmin(distances)
            
            # Check if the closest cluster is valid
            if closest_cluster in cluster_counts:
                break  # Exit the loop once a valid cluster is found
            
            # If not valid, continue searching
            distances[closest_cluster] = np.inf

        # merge the clusters
        labels[labels == cluster] = closest_cluster
        
        # update the cluster counts
        cluster_counts[closest_cluster] += count
        cluster_counts.pop(cluster)
        n_clusters -= 1
np.save("data/bert_clusters.npy",labels)

100%|██████████| 200/200 [00:17<00:00, 11.69it/s]


In [112]:
from scipy.spatial.distance import jaccard as jacc_score

hac =[]
for out in embeddings:#[np.argwhere(labels==20)]:
    #calculate the jaccard similarity between the first output and the rest
    jaccard = jacc_score(embeddings[0], out)
    hac.append(jaccard)

In [113]:
pd.set_option('display.max_colwidth', None)
ml_movies_description.iloc[0]

movieId                                                                                                                              1
title                                                                                                                 Toy Story (1995)
genres                                                                                     Adventure|Animation|Children|Comedy|Fantasy
imdbId                                                                                                                          114709
tmdbId                                                                                                                           862.0
id                                                                                                                              114709
description    A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room.
Name: 0, dtype: object

array([ 80, 104,  99, ...,  32,  20,  32], dtype=int32)

In [114]:
ml_movies_description.iloc[np.argsort(hac)[-5:][::-1]]

,movieId,title,genres,imdbId,tmdbId,id,description
3133,4675,Three Fugitives (1989),Action|Comedy,98471,31608.0,98471,"A reformed bank robber is taken hostage by a desperate man during a bank hold up, but is forced to go on the run with his captor when they're both mistakenly thought to be in cahoots."
817,1288,This Is Spinal Tap (1984),Comedy,88258,11031.0,88258,"Spinal Tap, one of England's loudest bands, is chronicled by film director Marty DiBergi on what proves to be a fateful tour."
5188,7951,Nightbreed (1990),Fantasy|Horror,100260,20481.0,100260,"A troubled young man is drawn to a mythical place called Midian where a variety of friendly monsters are hiding from humanity. Meanwhile, a sadistic serial killer is looking for a patsy."
803,1272,Patton (1970),Drama|War,66206,11202.0,66206,The World War II phase of the career of controversial American general George S. Patton.
5888,26110,36 Hours (1965),Thriller|War,57809,37684.0,57809,"Germans kidnap an American major and try to convince him that World War II is over, so that they can get details about the Allied invasion of Europe out of him."


In [121]:
ml_movies_description['bert_cluster'] = labels

ValueError: Length of values (6496) does not match length of index (51860)

In [120]:

ml_movies_description.iloc[np.argwhere(labels==80).flatten()]

,movieId,title,genres,imdbId,tmdbId,id,description
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,114709,A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room.
48,71,Fair Game (1995),Action,113010,11859.0,113010,"Max Kirkpatrick is a cop who protects Kate McQuean, a civil law attorney, from a renegade KGB team out to terminate her."
54,79,"Juror, The (1996)",Drama|Thriller,116731,9623.0,116731,A juror in a Mafia trial is forced to convince the other jurors to vote not guilty by an obsessive mob enforcer.
73,112,Rumble in the Bronx (Hont faan kui) (1995),Action|Adventure|Comedy|Crime,113326,33542.0,113326,A young man visiting and helping his uncle in New York City finds himself forced to fight a street gang and the mob with his martial art skills.
103,165,Die Hard: With a Vengeance (1995),Action|Crime|Thriller,112864,1572.0,112864,"John McClane and a Harlem store owner are targeted by German terrorist Simon in New York City, where he plans to rob the Federal Reserve Building."
...,...,...,...,...,...,...,...
6391,27695,Nicotina (2003),Action|Comedy|Drama,337930,11556.0,337930,A real-time dark comedy about a science geek who tangles with a clutch of Russian gangsters after he delivers them the wrong computer disk.
6398,27721,"Very Long Engagement, A (Un long dimanche de fiançailles) (2004)",Drama|Mystery|Romance|War,344510,2841.0,344510,"Tells the story of a young woman's relentless search for her fiancé, who has disappeared from the trenches of the Somme during World War One."
6420,27778,Ginger Snaps Back: The Beginning (2004),Fantasy|Horror,365265,12583.0,365265,"Set in 19th Century Canada, Brigette and her sister Ginger take refuge in a Traders' Fort which later becomes under siege by some savage werewolves."
6435,27812,Festival Express (2003),Documentary|Musical,372279,14770.0,372279,"The filmed account of a large Canadian rock festival train tour boasting major acts like Janis Joplin, The Grateful Dead and The Band."


In [118]:
ml_movies_description.iloc[np.argwhere(labels==3).flatten()].description

110                                   In a dystopian future, Joseph Dredd, the most famous Judge (a police officer with instant field judiciary powers), is convicted for a crime he did not commit and must face his murderous counterpart.
145                                   Hatch gets a special resuscitative medicine after "dying" in a car accident. His wife and daughter only get minor injuries. He gets strange nightmares such as killing a teen girl. Is it just dreams?
181                                  12-year-old Mathilda is reluctantly taken in by Léon, a professional assassin, after her family is murdered. An unusual relationship forms as she becomes his protégée and learns the assassin's trade.
245       A group of college seniors spending a weekend at a cottage for a psychological experiment called "fear therapy" find a human-sized wooden figure called Morty, which triggers everyone's deepest and darkest secrets and emotions.
305                       A pragmatic paleontologist

In [55]:
#jaccard for similarity between output 1 and outputs

#flatten the outputs
#calculate the jaccard similarity between the first output and the rest
for i in range(1, outputz.size(0)):
    #calculate the jaccard similarity between the first output and the rest
    jaccard = jacc_score(outputz[0], outputz[i])
    #take the mean of the jaccard similarity
    #add the jaccard similarity to the dataframe
    ml_movies_description[f'jaccard_{i}'] = jaccard

AttributeError: 'numpy.float64' object has no attribute 'cpu'

In [11]:

from datasketch import MinHash, MinHashLSH

In [16]:
lsh = MinHashLSH(threshold=0.7, num_perm=108)


In [29]:
import numpy as np
from sklearn.random_projection import SparseRandomProjection
from sklearn.metrics.pairwise import cosine_similarity

# Simulated BERT embeddings (replace this with your actual data)
embeddings = np.random.rand(10000, 768*108)

# Step 1: Reduce Dimensionality with Random Projection
n_components = 128  # Choose the reduced dimensionality
transformer = SparseRandomProjection(n_components=n_components)
reduced_embeddings = transformer.fit_transform(embeddings)

# Step 2: Implement LSH-like Bucketing (Simplified Example)

n_hash_planes = 16  # Number of hash planes
random_hyperplanes = np.random.randn(n_hash_planes, n_components)

# Compute binary hash (each vector is hashed based on the sign of the dot product)
binary_hash = (reduced_embeddings @ random_hyperplanes.T) > 0  # Boolean array
hash_buckets = binary_hash.astype(int)  # Convert to integer (0/1)

# Optional: Convert hash to integers for easier bucket indexing
bucket_indices = hash_buckets.dot(1 << np.arange(hash_buckets.shape[-1]))

# Step 3: Calculate Similarities within Buckets
query = reduced_embeddings[0]  # Select a query vector
query_hash = (query @ random_hyperplanes.T) > 0
query_bucket = query_hash.astype(int).dot(1 << np.arange(query_hash.shape[-1]))

# Find neighbors in the same bucket
bucket_mask = bucket_indices == query_bucket
neighbors = embeddings[bucket_mask]  # Embeddings in the same bucket
similarities = cosine_similarity([embeddings[0]], neighbors)  # Calculate similarity

print(f"Number of neighbors in the same bucket: {len(neighbors)}")


MemoryError: Unable to allocate 6.18 GiB for an array with shape (10000, 82944) and data type float64

In [17]:
# Insert each min-hash signature into the LSH
for idx, minhash in enumerate(outputs):
    lsh.insert(idx, minhash)  # Use the DataFrame index as the unique ID

AttributeError: 'Tensor' object has no attribute 'hashvalues'

In [ ]:


# Function to find similar items using LSH
def find_similar(description, lsh, num_perm=128):
    m = MinHash(num_perm=num_perm)
    for word in description.split():
        m.update(word.encode('utf8'))
    return lsh.query(m)

# save similaritities to dict
bands = {}
for idx in tqdm(range(len(outputs))):
    bands[idx] = find_similar(df_minhash['description'][idx], lsh)


In [10]:


def run_or_read_lsh(run: bool, df_minhash, threshold, num_perm=num_perm):

    if run:
        lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)

        # Insert each min-hash signature into the LSH
        for idx, minhash in enumerate(df_minhash['minhash']):
            lsh.insert(idx, minhash)  # Use the DataFrame index as the unique ID

        # Function to find similar items using LSH
        def find_similar(description, lsh, num_perm=128):
            m = MinHash(num_perm=num_perm)
            for word in description.split():
                m.update(word.encode('utf8'))
            return lsh.query(m)

        # save similaritities to dict
        bands = {}
        for idx in tqdm(range(len(df_minhash))):
            bands[idx] = find_similar(df_minhash['description'][idx], lsh)

        # save similarities to file as pickle
        with open(f'data/lsh_groups_{threshold}.pkl', 'wb') as f:
            pickle.dump(bands, f)
        print('LSH done and saved to lsh_groups.pkl')
    else:
        with open(f'data/lsh_groups_{threshold}.pkl', 'rb') as f:
            bands = pickle.load(f)
        print('LSH loaded from lsh_groups.pkl')
    return bands



NameError: name 'num_perm' is not defined

In [1]:
202

202